## __Virtual Internship Program by AINE AI__



### __Project : Classifying Covid-19 Positive and Negative Patients from X-Ray Images__

### __Aim: With the Chest X - Ray dataset, develop a Deep Learning Model to classify the X Rays of Healthy vs Corona Positive Patients__.

#####    -: __Vishal Parekh__

In [1]:
#Importing the required libraries
import pandas as pd
import numpy as np
import os
import cv2 as cv
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
CATEGORIES = ['Covid Negative', 'Covid Positive'] #Name of the Folders

In [3]:
DIRECTORY = r'Dataset' 

In [4]:
labels = [i for i in range(len(CATEGORIES))]
label_dict = dict(zip(CATEGORIES, labels)) #Creating dictionary for labels by merging two list

In [5]:
print(labels)
print(label_dict) # 0 for Negative and 1 for positive

[0, 1]
{'Covid Negative': 0, 'Covid Positive': 1}


### Resizing and Converting to Gray Images

In [6]:
img_size = 64
data = []
target = []

for category in CATEGORIES: #iterating through each folders
    path = os.path.join(DIRECTORY, category) #Path of the dataset i.e joining the path
    img_names = os.listdir(path)
    
    for img_name in img_names: # iterate through each image
        img_path = os.path.join(path, img_name)
        img = cv.imread(img_path) #reading each image files
        
        try:
            gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY) #Converting the image into gray scale
            resize = cv.resize(gray,(img_size, img_size))#resizing the image for all
            data.append(resize) #appending the data/features to the empty data list
            target.append(label_dict[category]) #appending the target labels into the empty target list
            #appending the image and the label into the list
        except Exception as e:
            print("Exception: ",e)
            #Exception for image corruption or unavailable and continue
    

In [7]:
data = np.array(data) / 255.0 #Converting to numpy array and normalize the data to 0 and 1

In [8]:
data.shape #2271 images of size 100X100

(2513, 64, 64)

In [9]:
x = data.copy()

In [10]:
y = target.copy() #target containing the labels of the image i.e 0,1 for negative and normal images

In [11]:
import pickle 

In [12]:
pickle.dump(x, open('x.pkl','wb')) #save the processed variable to our local machine
pickle.dump(y, open('y.pkl','wb')) #wb is write binary

In [13]:
from keras.utils import np_utils

In [14]:
X = pickle.load(open('x.pkl', 'rb')) #loading the pickle files
y = pickle.load(open('y.pkl', 'rb'))

In [15]:
y = np.array(y)

In [16]:
X = X.reshape(-1, 64, 64, 1) # for CNN model we need to reshape it to 4D array (batchsize,height,width and depth)

In [17]:
X.shape

(2513, 64, 64, 1)

In [18]:
from keras.models import Sequential,Model
from keras.layers import Dense,Dropout,Flatten
from keras.layers import Conv2D,Activation,MaxPooling2D
from keras.utils import normalize
from keras import Input
from keras.callbacks import ModelCheckpoint

In [19]:
model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape = X.shape[1:])) #In CNN. input shpae has to be given in the first layer
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2))) 
model.add(Conv2D(32, (3, 3))) #2nd CNN layer with 32 filter with 3,3
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Flatten()) # Converting to 1D matrix
# model.add(Dropout(0.5))
model.add(Dense(128)) #128 neurons in the hidden layers
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation = 'softmax')) #Final layers of 2 layers same as the output to predict
model.compile(optimizer = 'adam', loss = "sparse_categorical_crossentropy", metrics=['accuracy'])

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 64)        640       
_________________________________________________________________
activation (Activation)      (None, 62, 62, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        18464     
_________________________________________________________________
activation_1 (Activation)    (None, 29, 29, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6272)              0

In [21]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1)

In [22]:
history = model.fit(X_train,y_train,epochs=20,validation_split=0.1)

Epoch 1/20
64/64 [==============================] - 11s 152ms/step - loss: 0.6490 - accuracy: 0.5760 - val_loss: 0.2972 - val_accuracy: 0.9471
Epoch 2/20
64/64 [==============================] - 9s 134ms/step - loss: 0.2317 - accuracy: 0.9318 - val_loss: 0.1680 - val_accuracy: 0.9295
Epoch 3/20
64/64 [==============================] - 9s 134ms/step - loss: 0.1302 - accuracy: 0.9594 - val_loss: 0.0833 - val_accuracy: 0.9648
Epoch 4/20
64/64 [==============================] - 9s 137ms/step - loss: 0.1108 - accuracy: 0.9674 - val_loss: 0.0718 - val_accuracy: 0.9956
Epoch 5/20
64/64 [==============================] - 9s 139ms/step - loss: 0.0737 - accuracy: 0.9741 - val_loss: 0.0429 - val_accuracy: 0.9912
Epoch 6/20
64/64 [==============================] - 8s 132ms/step - loss: 0.0701 - accuracy: 0.9710 - val_loss: 0.0541 - val_accuracy: 0.9780
Epoch 7/20
64/64 [==============================] - 8s 132ms/step - loss: 0.0614 - accuracy: 0.9768 - val_loss: 0.0557 - val_accuracy: 0.9868
Epoch

In [23]:
scores = model.evaluate(X_test, y_test) #Evaluating the model with the help of test datas
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

8/8 [==============================] - 0s 31ms/step - loss: 0.2122 - accuracy: 0.9841

accuracy: 98.41%


In [24]:
model.save('cnnmodel.h5')